In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [ ]:
import tensorflow as tf
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(color_codes=True)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import cross_val_score, KFold

In [ ]:
cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [ ]:
df=pd.read_csv('2020_Competition_Training (1).csv')

In [ ]:
df

,person_id_syn,transportation_issues,src_platform_cd,sex_cd,est_age,smoker_current_ind,smoker_former_ind,lang_spoken_cd,mabh_seg,cci_score,dcsi_score,fci_score,hcc_weighted_sum,betos_d1c_pmpm_ct,betos_d1d_pmpm_ct,betos_m1b_pmpm_ct,betos_m2c_pmpm_ct,betos_m5b_pmpm_ct,betos_m5c_pmpm_ct,betos_m5d_pmpm_ct,betos_o1a_pmpm_ct,betos_o1b_pmpm_ct,betos_o1e_pmpm_ct,betos_o1g_pmpm_ct,betos_t1a_pmpm_ct,betos_t1b_pmpm_ct,betos_t1e_pmpm_ct,betos_t1h_pmpm_ct,betos_t2a_pmpm_ct,betos_y2_pmpm_ct,bh_adtp_ind,bh_aoth_ind,bh_bipr_ind,bh_cdal_ind,bh_cdsb_ind,bh_cdto_ind,bh_dema_ind,ccsp_014_ind,ccsp_020_ind,ccsp_021_ind,...,submcc_inf_othr_ind,submcc_dig_liv_ind,submcc_gus_kub_ind,submcc_brn_acc_ind,submcc_dia_othr_ind,submcc_cer_hem_ind,submcc_ben_othr_ind,submcc_inj_drug_ind,submcc_rsk_fh/ho_ind,betos_y2_ind,rx_gpi2_52_ind,submcc_inf_myco_ind,submcc_rar_par_ind,submcc_rsk_an_ind,submcc_rar_hem_ind,rx_gpi2_48_ind,betos_m1b_ind,rx_gpi2_57_ind,rx_gpi2_09_ind,rx_gpi2_55_ind,submcc_sor_eye_ind,submcc_hiv_kapo_ind,submcc_cad_mi_ind,rx_gpi2_73_ind,rx_gpi2_15_ind,rx_gpi2_70_ind,rx_gpi2_88_ind,submcc_res_alg_ind,submcc_can_lymp_ind,submcc_mus_arth_ind,submcc_rar_scl_ind,rx_gpi2_74_ind,rx_gpi2_89_ind,rx_gpi2_96_ind,submcc_rsk_obe_ind,rx_gpi2_22_ind,submcc_rsk_synx_ind,submcc_rsk_coag_ind,submcc_rsk_othr_ind,submcc_rsk_chol_ind
0,0002MOb79ST17bLYAe46eIc2,0,EM,F,62,1,0,ENG,UNK,3.0,1,2,0,0.0,0.0,1.162658,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0004cMOS6bTLf34Y7AIca8f3,0,EM,F,59,1,0,ENG,C2,1.0,2,5,3,0.0,0.0,1.155124,0.0,0.0,0.085593,0.085873,0.000000,0.0,0.000000,0.000000,0.175142,0.000000,0.000000,0.090370,0.000000,0.0,0,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,000536M9O3ST98LaYaeA29Ia,1,EM,F,63,0,0,ENG,UNK,3.0,0,6,0,0.0,0.0,0.333333,0.0,0.0,0.133333,0.083333,1.250000,0.0,0.166667,0.083333,0.000000,0.166667,0.083333,0.250000,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0009bMO9SfTLYe77A51I4ac3,0,EM,M,75,0,0,ENG,H6,3.0,0,2,0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.083333,0.000000,0.0,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
4,000M7OeS66bTL8bY89Aa16Ie,0,EM,M,51,1,0,ENG,UNK,3.0,1,4,0,0.0,0.0,0.083333,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.083333,0.000000,0.166667,0.166667,0.000000,0.250000,0.000000,0.0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69567,ffe33MOS25dTf027LaY7A5I3,0,EM,F,72,1,0,ENG,H7,5.0,2,2,2,0.0,0.0,1.250000,0.0,0.0,0.166667,0.083333,0.000000,0.0,0.250000,0.000000,0.250000,0.500000,0.000000,0.916667,0.250000,0.0,0,0,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
69568,fff1M4O1cfST49LY464A2Ieb,0,EM,M,75,0,0,ENG,C4,9.0,3,2,14,0.0,0.0,1.668168,0.0,0.0,0.000000,0.332639,0.000000,0.0,0.252794,0.000000,0.449687,0.735747,0.000000,0.627186,0.346925,0.0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
69569,fff5MO7e401STLYcAd8e581I,0,EM,M,76,0,0,ENG,H6,6.0,2,4,7,0.0,0.0,0.666667,0.0,0.0,0.083333,0.000000,0.083333,0.0,0.000000,0.083333,0.416667,0.166667,0.000000,0.166667,0.000000,0.0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
69570,fffMc37OSfTLfY7853dfA09I,0,LV,M,67,0,1,ENG,H6,2.0,0,

In [ ]:
# Use 0 to replace 1.1
df['pdc_ast'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_cvd'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_dep'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_dia'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_hf'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_ht'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_lip'].apply(lambda x: 1 if x==1.1 else x)
df['pdc_ost'].apply(lambda x: 1 if x==1.1 else x)
# impute NA with median value
df=df.fillna(df.median())

In [ ]:
df.drop(['person_id_syn','hedis_ami','zip_cd', 'cnty_cd', 'state_cd'],axis=1, inplace=True)

In [ ]:
# convert categorical feature
no_hoc_cat=['rucc_category','hedis_cmc_ldc_c_control','hedis_cmc_ldc_c_screen',
            'hedis_dia_eye','hedis_dia_hba1c_ge9','hedis_dia_hba1c_test','hedis_dia_ldc_c_control',
            'hedis_dia_ldc_c_screen','hedis_dia_ma_nephr','cms_ra_factor_type_cd',
            'mabh_seg','lang_spoken_cd','sex_cd','src_platform_cd',
            'cons_hhcomp','cons_homstat','cons_cmys']
df_1hot=pd.DataFrame()
for s in no_hoc_cat:
    df_temp = pd.get_dummies(df[s], prefix = s)
    df_1hot = pd.concat([df_1hot,df_temp], axis=1)
df=pd.concat([df, df_1hot], axis=1)
df.drop(no_hoc_cat, axis=1, inplace=True) 

In [ ]:
x=df.loc[:,['cms_ma_risk_score_nbr', 'med_er_visit_ct_pmpm', 'dcsi_score', 'credit_hh_agencyfirstmtg_new',\
        'cons_n65p_y', 'betos_m1b_pmpm_ct', 'credit_bal_autobank', 'cms_partd_ra_factor_amt', 'rx_mail_pmpm_ct',\
        'credit_bal_mtgcredit_new', 'credit_hh_autobank', 'ccsp_220_ind', 'bh_bipr_ind', 'cms_rx_risk_score_nbr',\
        'credit_hh_studentloan_60dpd', 'ccsp_239_ind', 'cms_ra_factor_type_cd_CF', 'rx_gpi2_49_pmpm_ct', 'cms_low_income_ind', \
        'cons_homstat_Y', 'total_outpatient_visit_ct_pmpm', 'submcc_men_othr_pmpm_ct', 'submcc_ner_othr_pmpm_ct', 'pdc_dep',\
        'betos_t1a_ind', 'cms_disabled_ind', 'fci_score', 'bh_cdto_ind', 'cms_ra_factor_type_cd_CN', 'cons_hhcomp_B',\
        'submcc_ner_infl_ind', 'rev_cms_ambul_ind', 'submcc_trm_skul_pmpm_ct', 'rx_gpi2_87_pmpm_ct', 'hlth_pgm_slvrsnkr_pct_par',\
        'submcc_men_alco_ind', 'cmsd2_men_mad_ind', 'cms_ra_factor_type_cd_CP', 'est_age']].values

In [ ]:
#dummies = pd.get_dummies(df['transportation_issues']) # Classification
#species = dummies.columns
#y = dummies.values

In [ ]:
y=df['transportation_issues']
y

0        0
1        0
2        1
3        0
4        0
        ..
69567    0
69568    0
69569    0
69570    0
69571    1
Name: transportation_issues, Length: 69572, dtype: int64

##Model Training

In [ ]:
# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

# Build neural network
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
#model.add(Dense(25, activation='relu')) # Hidden 3
#model.add(Dense(25, activation='relu')) # Hidden 4
model.add(Dense(y.shape[1],activation='softmax')) # Output
model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-8, patience=5, 
        verbose=1, mode='auto', restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
        callbacks=[monitor],verbose=2,epochs=1000)

Epoch 1/1000
1631/1631 - 3s - loss: 24.2998 - val_loss: 18.9392
Epoch 2/1000
1631/1631 - 2s - loss: 4.8132 - val_loss: 2.1142
Epoch 3/1000
1631/1631 - 2s - loss: 3.7149 - val_loss: 5.0348
Epoch 4/1000
1631/1631 - 2s - loss: 2.7471 - val_loss: 6.7844
Epoch 5/1000
1631/1631 - 2s - loss: 2.2242 - val_loss: 2.6862
Epoch 6/1000
1631/1631 - 2s - loss: 1.6730 - val_loss: 1.0369
Epoch 7/1000
1631/1631 - 2s - loss: 1.4973 - val_loss: 0.7597
Epoch 8/1000
1631/1631 - 2s - loss: 1.2974 - val_loss: 0.9614
Epoch 9/1000
1631/1631 - 2s - loss: 0.9863 - val_loss: 0.4990
Epoch 10/1000
1631/1631 - 2s - loss: 0.8777 - val_loss: 1.3352
Epoch 11/1000
1631/1631 - 2s - loss: 0.7489 - val_loss: 0.9969
Epoch 12/1000
1631/1631 - 2s - loss: 0.6048 - val_loss: 0.7600
Epoch 13/1000
1631/1631 - 2s - loss: 0.5869 - val_loss: 0.3766
Epoch 14/1000
1631/1631 - 2s - loss: 0.5181 - val_loss: 0.4019
Epoch 15/1000
1631/1631 - 2s - loss: 0.4879 - val_loss: 0.3857
Epoch 16/1000
1631/1631 - 2s - loss: 0.4686 - val_loss: 0.4735

In [ ]:
from sklearn.metrics import accuracy_score

pred = model.predict(x_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Accuracy: {correct}")

Accuracy: 0.8555165871327546


In [ ]:
m = tf.keras.metrics.AUC(num_thresholds=200)
m.update_state(expected_classes, pred[:,1])
m.result().numpy()

0.71945155

In [ ]:
y.value_counts(normalize=True)

0    0.853432
1    0.146568
Name: transportation_issues, dtype: float64

### Change Parameters

In [ ]:
# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

# Build neural network
model = Sequential()
model.add(Dense(128, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(64, activation='relu')) # Hidden 2
model.add(Dense(32, activation='relu')) # Hidden 3
model.add(Dense(8, activation='relu')) # Hidden 4
model.add(Dense(1,activation='sigmoid')) # Output
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

#monitor = EarlyStopping(monitor='val_loss', min_delta=1e-8, patience=5,verbose=1, mode='auto', restore_best_weights=True)
#callbacks=[monitor],
model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=2,epochs=5)

Epoch 1/5
1631/1631 - 3s - loss: 0.6596 - accuracy: 0.8514 - val_loss: 0.4327 - val_accuracy: 0.8563
Epoch 2/5
1631/1631 - 2s - loss: 0.4561 - accuracy: 0.8524 - val_loss: 0.4168 - val_accuracy: 0.8563
Epoch 3/5
1631/1631 - 2s - loss: 0.4394 - accuracy: 0.8525 - val_loss: 0.4262 - val_accuracy: 0.8563
Epoch 4/5
1631/1631 - 2s - loss: 0.4448 - accuracy: 0.8525 - val_loss: 0.4151 - val_accuracy: 0.8563
Epoch 5/5
1631/1631 - 2s - loss: 0.4199 - accuracy: 0.8524 - val_loss: 0.4008 - val_accuracy: 0.8563


In [ ]:
model.predict(x_train)

array([[0.14684424],
       [0.14684424],
       [0.14684424],
       ...,
       [0.14684424],
       [0.14684424],
       [0.14684424]], dtype=float32)

### This shows that our model always predicts the same number. We are working on this.

In [ ]:
from sklearn.metrics import accuracy_score

pred = model.predict(x_test)
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y_test,axis=1)
correct = accuracy_score(expected_classes,predict_classes)
print(f"Accuracy: {correct}")

In [ ]:
m = tf.keras.metrics.AUC(num_thresholds=200)
m.update_state(expected_classes, pred[:,1])
m.result().numpy()